In [2]:
import numpy as np
import gensim

In [1]:
import sys
if "../modules/" not in sys.path: sys.path.append ("../modules/")
from semshift import measures, embeddings, alignment

In [3]:
early_model = embeddings.TrainedModel("/hg191/corpora/legaldata/sc-docs/0.model")
later_model = embeddings.TrainedModel("/hg191/corpora/legaldata/sc-docs/7.model")

In [5]:
aligned = alignment.smart_procrustes_align_gensim(early_model.m, later_model.m)

In [10]:
vocab = set (list (early_model.m.wv.vocab.keys()))

In [11]:
linguistic_drift_scores = {w: measures.HamiltonMeasures.linguistic_drift(early_model.m, later_model.m, w) for w in vocab}

In [12]:
k=25
cultural_shift_scores = {w: measures.HamiltonMeasures.cultural_shift (early_model.m, later_model.m, w, k) for w in vocab}

/nethome/ssoni30/venvs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [13]:
namefiles = ["/hg191/corpora/legaldata/sc-docs/names.neural", "/hg191/corpora/legaldata/sc-docs/names.tagging"]

names = set ()
for namefile in namefiles:
    with open (namefile) as fin:
        more_names = {line.strip() for line in fin}
    names = names | more_names

In [23]:
assert (len (linguistic_drift_scores) == len (cultural_shift_scores))
print (len (linguistic_drift_scores))
print (len(names & set (list (linguistic_drift_scores.keys()))))
print (len (linguistic_drift_scores) - len(names & set (list (linguistic_drift_scores.keys()))))

91661
47778
43883


In [26]:
with open ("/hg191/corpora/legaldata/sc-docs/vocab.linguistic_drift", "w") as fout:
    for word,score in sorted(linguistic_drift_scores.items(), key=lambda x:x[1], reverse=True):
        if word not in names:
            fout.write ("{0}\t{1}\n".format (word,score))

In [27]:
with open ("/hg191/corpora/legaldata/sc-docs/vocab.cultural_shift", "w") as fout:
    for word,score in sorted (cultural_shift_scores.items(), key=lambda x:x[1], reverse=True):
        if word not in names:
            fout.write ("{0}\t{1}\n".format (word,score))